In [ ]:
import seaborn.objects as so
import json
import pandas as pd
import glob
from pandas import DataFrame
import re
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

In [ ]:
file_patter = re.compile("../results/([^/]+)/([^/]+)/([^/]+).json")


def get_results_into_df(file_path: str) -> DataFrame:
    match = file_patter.fullmatch(file_path)
    if match is None:
        raise

    with open(file_path) as file:
        data = json.load(file)
        df = pd.json_normalize(data)
        df["tag"] = match.group(1)
        df["timestamp"] = match.group(2)
        df["size"] = int(match.group(3))
        return df


# results/<tag>/<timestamp>/<size>.json
results_df = pd.concat(map(get_results_into_df, glob.glob("../results/*/*/*.json")))

In [ ]:
results_df[['tag', 'timestamp']].groupby(['tag', 'timestamp']).first()

In [ ]:
df = results_df[(results_df['tag'] == 'xdp-bench-tx') & (results_df['timestamp'] == '2025-04-23T17:05:29-03:00') & (results_df['size'] == 64)]

In [ ]:
df[["iteration", "expected_duration_ms", "timeout_ms", "actual_duration_ms", "stopped_early", "rx_delay_ms", "rate", "stats.global.tx_bps", "stats.global.rx_bps", "stats.total.opackets", "stats.total.ipackets", "lost", "lost_percentage"]]

In [ ]:
df

In [ ]:
for column in df.columns.array.tolist():
    try:
        plot = so.Plot(df, x='iteration', y=column).add(so.Dot()).label(title=column)
        plot.show()
    except Exception:
        print("error in collumn" + column)


In [ ]:
#(so.Plot(df, x='iteration', y='stats.global.rx_core_pps', color="tag")
#.add(so.Dot()))

In [ ]:
#for column in df.columns.array.tolist():
#    try:
#        plot = so.Plot(df, x='iteration', y=column).add(so.Dot())
#        plot.show()
#    except Exception:
#        print("error in collumn" + column)